<h2 style="text-align: center; background-color: #4d648d; font-family:Arial; color: white; padding: 13px; line-height: 1; border-radius:10px"> 03_02 Adaptation et fine-tuning </h2>

Le fine-tuning :  


*   Spécialiser le modèle sur un domaine (ex : médical, juridique, ...)  
*   Adapter à une tâche précise : classification, génération, question/réponse, ...  
*   Corriger le style, le ton ou les biais pour un public, yn usage particulier

Comment ?


1.   Full fine-tuning	: ajustement des poids
2.   LoRA (Low-Rank Adaptation) : ajout de couches adaptatives
3.   On entraîne sur des paires "instruction + réponse"



In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Chargement du tokenizer + modèle
tokenizer = AutoTokenizer.from_pretrained(model_name, token="XXXXXXXXXXXXXXXXXXXXXXx")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", token="XXXXXXXXXXXXXXXXXXXXXXx")

def get_embedding_mistral(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    with torch.no_grad():
        outputs = model.model(**inputs, output_hidden_states=True)
        last_hidden = outputs.hidden_states[-1]  # dernière couche
        embedding = last_hidden.mean(dim=1).squeeze().cpu().numpy()
    return embedding

texts = [
    "J'adore ce produit !", "C'est très mauvais", "C'était moyen", "Exceptionnel", "Nul", "Correct"
]
labels = ["positif", "négatif", "neutre", "positif", "négatif", "neutre"]

X = np.array([get_embedding_mistral(text) for text in texts])
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


              precision    recall  f1-score   support

      neutre       0.00      0.00      0.00       0.0
     négatif       0.00      0.00      0.00       1.0
     positif       0.00      0.00      0.00       1.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_